In [1]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from requests_html import HTMLSession

# Отключение некоторых лишних предупреждений
import warnings
warnings.filterwarnings("ignore")

In [2]:
#pd.read_html('https://www.basketball-reference.com/teams/LAL/2023.html')

In [5]:
# нашли список аббревеатур команд NBA с википедии
team_list = pd.read_html("https://en.m.wikipedia.org/wiki/Wikipedia:WikiProject_National_Basketball_Association/National_Basketball_Association_team_abbreviations")[0]

# обратились к нулевому столбцу таблицы и переделали ее в list
teams_15_23 = team_list.iloc[1:, 0].to_list()

# команды со временем меняют свою дислокацию и из-за этого меняют название
# за 21 век таких изменений было три. дальше будет код, где мы вручную прописали замену аббревеатур
# заменяем вручную несовпавшие аббревеатуры команд из таблички с википедии и аббревеатур с URL сайта, 
# которые были выявлены после первых попыток парсинга

teams_15_23[1], teams_15_23[3], teams_15_23[23] = 'BRK', 'CHO', 'PHO'

# создадим лист с аббревеатурами команд до сезона 07-08, когда команды еще базировались в городах, где были основаны 
teams_00_08 = teams_15_23.copy()
old_value_0, old_value_1, old_value_2 = 'OKC', 'BRK', 'CHO'
new_value_0, new_value_1, new_value_2 = 'SEA', 'NJN', 'CHA'

index_0 = teams_00_08.index(old_value_0)
index_1 = teams_00_08.index(old_value_1)
index_2 = teams_00_08.index(old_value_2)
teams_00_08[index_0], teams_00_08[index_1], teams_00_08[index_2] = new_value_0, new_value_1, new_value_2

# после сезона 08-09 года команда Seattle SuperSonics переехала в Оклахому и стала называться Oklahoma City Thunder 
teams_09_12 = teams_00_08.copy()
old_value_3 = 'SEA'
new_value_3 = 'OKC'
index_3 = teams_09_12.index(old_value_3)
teams_09_12[index_3] = new_value_3 

# после сезона 13-14 года команда Charlotte Bobcats поменяли название и стали Charlotte Hornets
teams_13_14 = teams_09_12.copy()
old_value_4 = 'CHA'
new_value_4 = 'CHO'
index_4 = teams_13_14.index(old_value_4)
teams_13_14[index_4] = new_value_4
print(teams_00_08, teams_09_12, teams_13_14, teams_15_23, sep='\n')

['ATL', 'NJN', 'BKN', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'SEA', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']
['ATL', 'NJN', 'BKN', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']
['ATL', 'NJN', 'BKN', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']
['ATL', 'BRK', 'BKN', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']


In [4]:
import time

years = [int(i) for i in range(2008, 2024)]

data_00_08 = dict()
data_09_12 = dict()
data_13_14 = dict()
data_15_23 = dict()

def get_key(team, year):
    return f'{team}_{year}'

def get_element_00_08(team, year):
    return data.get(get_key(team, year))

for year in years:
    if year < 2009:
        for team in teams_00_08:
            link = f'https://www.basketball-reference.com/teams/{team}/{year}.html'
            
            print(get_key(team, year))
            data_00_08.update({get_key(team,year): pd.read_html(link)[0]})
            time.sleep(5)
    else:
        if year < 2013:
            for team in teams_09_12:
                link = f'https://www.basketball-reference.com/teams/{team}/{year}.html'
                
                print(get_key(team, year))
                data_09_12.update({get_key(team,year) : pd.read_html(link)[0]})
                time.sleep(5)
        else:
            if year < 2015:
                for team in teams_13_14:
                    link = f'https://www.basketball-reference.com/teams/{team}/{year}.html'
                    
                    print(get_key(team, year))
                    data_13_14.update({get_key(team,year) : pd.read_html(link)[0]})
                    time.sleep(5)
            else:
                for team in teams_15_23:
                    link = f'https://www.basketball-reference.com/teams/{team}/{year}.html'
                    
                    print(get_key(team, year))
                    data_15_23.update({get_key(team,year) : pd.read_html(link)[0]})
                    time.sleep(5)

ATL_2008
NJN_2008
BKN_2008


HTTPError: HTTP Error 404: Not Found

In [ ]:
data_00_08

In [ ]:
headers = {'User-Agent': UserAgent().random}
url = ''
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

team_stats = soup.find()

In [ ]:
asession = HTMLSession()

r = asession.get()

r.html.arender()
resp = r.html.raw_html
soup = BeautifulSoup(resp, 'html.parser')

team_stats = soup.find()